In [60]:
#Import Libraries and Packages

import os
import json

# keras imports
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
import keras.utils as image
from keras.models import Model


import h5py
import numpy as np
import pickle as pk

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
#Loading Models for various checks
first_check = VGG16(weights='imagenet')
second_check = pk.load(open("C:/Users/avish/Desktop/cdd/car_damage_check/classifier.pickle", 'rb')) #damaged vs whole
third_check = pk.load(open("C:/Users/avish/Desktop/cdd/car_damage_check/FRS/classifier.pickle", 'rb')) # Front Rear Side
fourth_check = pk.load(open("C:/Users/avish/Desktop/cdd/car_damage_check/Severity/classifier.pickle", 'rb')) # Minor Moderate Severe

In [63]:
# Loading  valid categories for identifying cars using VGG16
with open('cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [64]:
cat_counter

Counter({('n04252225', 'snowplow'): 7.086045803967863,
         ('n03478589', 'half_track'): 0.8668184897396713,
         ('n04461696', 'tow_truck'): 18.77435230812989,
         ('n03126707', 'crane'): 0.1037683803588152,
         ('n03417042', 'garbage_truck'): 1.415972523856908,
         ('n02701002', 'ambulance'): 5.562831717485096,
         ('n03977966', 'police_van'): 10.421007923898287,
         ('n03345487', 'fire_engine'): 2.669069046038203,
         ('n03670208', 'limousine'): 53.60916757641826,
         ('n03594945', 'jeep'): 30.023131960071623,
         ('n03930630', 'pickup'): 65.0029446555709,
         ('n04467665', 'trailer_truck'): 3.2392259969492443,
         ('n02814533', 'beach_wagon'): 67.81617850526527,
         ('n03770679', 'minivan'): 132.04886488290504,
         ('n02974003', 'car_wheel'): 17.176703382167034,
         ('n03100240', 'convertible'): 91.413807330071,
         ('n03459775', 'grille'): 18.05060733258142,
         ('n04285008', 'sports_car'): 116.2507

In [65]:
# shortlisting top 27 Categories that VGG16 stores for cars (Can be altered for less or more)
cat_list  = [k for k, v in cat_counter.most_common()[:27]]
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

In [66]:
CLASS_INDEX = None

In [67]:
CLASS_INDEX_PATH = "C:/Users/avish/Desktop/Major2/imagenet_class_index.json"

In [68]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [69]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [70]:
def car_categories_check(img_224, model):
    print ("Validating that this is a picture of your car...")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print ("Car Check Passed!!!")
            print ("\n")
            return True 
    return False 

In [71]:
def car_damage_check(classifier):
    print ("Validating that damage exists...")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print ("Validation complete - proceeding to location and severity determination")
        print ("\n")
        return True 
    else:
        return False

In [72]:
def location_assessment(classifier):
    print ("Validating the damage area - Front, Rear or Side")
    base_model = first_check
    train_labels = ['Front Damage', 'Rear Damage', 'Side Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car is damaged at - " + train_labels[preds[0]])
    print ("Location assesment complete")
    print("\n")

In [79]:
def severity_assessment(classifier):
    print ("Validating the Severity...")
    base_model = first_check
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car damage impact is - " + train_labels[preds[0]])
    print ("Severity assesment complete")
    print ("\n")
    print ("Thank you for using the our software!!!")

In [80]:

def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)

            if g1 is False:
                print ("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            g2 = car_damage_check(second_check)

            if g2 is False:
                print ("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            x = location_assessment(third_check)
            y = severity_assessment(fourth_check)
            break
        except:
            print ("Image not accessible. Please try again.")
            break

In [85]:
img_path = "C:/Users/avish/Desktop/Major2/data2a/validation/02-side/0034.JPEG"

In [86]:
engine(img_path)

Validating that this is a picture of your car...
1/1 [==============================] - 0s 244ms/step
Car Check Passed!!!


Validating that damage exists...
1/1 [==============================] - 1s 574ms/step
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
1/1 [==============================] - 1s 745ms/step
Your Car is damaged at - Side Damage
Location assesment complete


Validating the Severity...
1/1 [==============================] - 1s 657ms/step
Your Car damage impact is - Moderate Damage
Severity assesment complete


Thank you for using the our software!!!
